# Example of using the Borehole3D class

## Imports

In [1]:
from striplog import Component, Decor, Legend, Lexicon, Striplog
from core.omf import Borehole3D, striplog_legend_to_omf_legend 
from utils.lexicon import Lexicon_FR_updated as lexicon_fr
from utils.io import boreholes_from_files, gdf_viewer
from core.core import Project
from core.orm import BoreholeOrm, PositionOrm, Base 

import matplotlib.pyplot as plt
from os import remove
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

## Reading of CSV files in a dir

In [2]:
save_dir = ROOT_DIR+'/CF_data/Donnees_fusionnees/'

In [3]:
df = pd.read_csv(save_dir+'Pilote_test.csv', sep=',')

In [4]:
gdf_viewer(df)

Rows : 86, columns : 10


interactive(children=(IntSlider(value=10, description='rows', max=86, min=10, readout=False), IntSlider(value=…

## Data Checking

In [5]:
df.drop(index=df.query('X!=X').index, inplace=True)

In [6]:
df.query('ID=="201"')

,ID,Ep_litho,X,Y,Z,Long_for,Litho_base,Litho_top,Diam_for,Description
0,201,0.6,152890.245758,122596.47426,101.926886,2.4,1.2,0.6,75.0,Remblais avec briques de construction couleur ...
1,201,1.2,152890.245758,122596.47426,101.926886,2.4,2.4,1.2,75.0,Briquaille de déchets de construction et galet...


## Create Boreholes from dataframes or files

In [7]:
Lexicon.default()

{'lithology': ['overburden', 'sandstone', 'siltstone', 'shale', 'conglomerate', 'mudstone', 'limestone', 'dolomite', 'salt', 'halite', 'anhydrite', 'gypsum', 'sylvite', 'clay', 'mud', 'silt', 'sand', 'gravel', 'boulders'], 'modifier': ['silty', 'sandy', 'shale?y', 'muddy', 'pebbly', 'gravell?y'], 'amount': ['streaks?', 'veins?', 'stringers?', 'interbed(?:s|ded)?', 'blotch(?:es)?', 'bands?', 'fragments?', 'impurit(?:y|ies)', 'abundant', 'minor', 'some', 'rare', 'flakes?', 'trace', '[-.\\d]+%', '[-.\\d]+pc', '[-.\\d]+per ?cent'], 'grainsize': ['vf(?:-)?', 'f(?:-)?', 'm(?:-)?', 'c(?:-)?', 'vc', 'very fine(?: to)?', 'fine(?: to)?', 'medium(?: to)?', 'coarse(?: to)?', 'very coarse', 'v fine(?: to)?', 'med(?: to)?', 'med.(?: to)?', 'v coarse', 'grains?', 'granules?', 'pebbles?', 'cobbles?', 'boulders?'], 'colour': ['red(?:dish)?', 'gray(?:ish)?', 'grey(?:ish)?', 'black(?:ish)?', 'whit(?:e|ish)', 'blu(?:e|ish)', 'purpl(?:e|ish)', 'yellow(?:ish)?', 'green(?:ish)?', 'brown(?:ish)?', 'light', 'd

In [8]:
bhs_df=[df] # list of dataframes

In [9]:
boreholes, components, link_intv_comp = boreholes_from_files(bhs_df, verbose=True,
                                                litho_field='Description',
                                                lexicon='fr', #Lexicon.default(),
                                                diam_field='Diam_for', 
                                                litho_top_field='Litho_top',
                                                litho_base_field='Litho_base',
                                                thick_field='Ep_litho')


Dataframe 0 processing...
|__ID:'201'
Summary : [Striplog(2 Intervals, start=0.6, stop=2.4)]
{0: {'description': 'Remblais avec briques de construction couleur orange, gris, verdâtre  ', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f803f812940>, 'base': <core.orm.PositionOrm object at 0x7f808c59e670>}, 1: {'description': "Briquaille de déchets de construction et galets roulés - présence d'eau - légère odeur de naphtalène - Bloqué à 2,4 m sur caillasse (et béton ?)", 'interval_number': 1, 'top': <core.orm.PositionOrm object at 0x7f803f7e9340>, 'base': <core.orm.PositionOrm object at 0x7f803f7e9400>}}

|__ID:'205'
Error : No lithology matching with 'Couche de boue en surface liée à l'intervention de Géosonda' in given lexicon
Summary : [Striplog(7 Intervals, start=0.0, stop=4.8)]
{2: {'description': 'white sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f803f7dcbb0>, 'base': <core.orm.PositionOrm object at 0x7f803f7dc4f0>}, 3: {'description': 'L

Error : No lithology matching with 'Gouge vide' in given lexicon
Summary : [Striplog(7 Intervals, start=0.0, stop=4.8)]
{41: {'description': 'Remblais caillouteux gris foncé humide', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f803f792ac0>, 'base': <core.orm.PositionOrm object at 0x7f803f735ac0>}, 42: {'description': 'Remblais brun matrice sableuse et remblais de déchets de construction brun/beige', 'interval_number': 1, 'top': <core.orm.PositionOrm object at 0x7f803f735940>, 'base': <core.orm.PositionOrm object at 0x7f803f738a60>}, 43: {'description': 'Remblais noirs gravilloneux ', 'interval_number': 2, 'top': <core.orm.PositionOrm object at 0x7f803f7384c0>, 'base': <core.orm.PositionOrm object at 0x7f803f738a30>}, 44: {'description': 'Matériaux gravillonneux noirs saturés gras - présence de schistes et scories - légère odeur vers 2 m dans strate saturée ', 'interval_number': 3, 'top': <core.orm.PositionOrm object at 0x7f803f738910>, 'base': <core.orm.PositionOrm 

/home/yanathan/Projects/GSDMA/utils/io.py:519: SAWarning: relationship 'ComponentOrm.intervals' will copy column Components.id to column Linkintervalcomponent.comp_id, which conflicts with relationship(s): 'IntervalOrm.components' (copies Components.id to Linkintervalcomponent.comp_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   The 'overlaps' parameter may be used to remove this warning.
  boreholes.append(BoreholeOrm(id=bh_name))
/home/yanathan/Projects/GSDMA/utils/io.py:519: SAWarning: relationship 'ComponentOrm.intervals' will copy column Intervals.id to column Linkintervalcomponent.int_id, which conflicts with relationship(s): 'IntervalOrm.components' (copies Intervals.id to

In [10]:
litho_list = []
for i, c in components.items():
    if not hasattr(c, 'lithology'):
        c.lithology = 'sable'
        c.colour = 'rouge'
    elif c.lithology == 'sand':
        c.lithology = 'sable'
        c.colour = 'blanc'
    if c.lithology not in litho_list:
        litho_list.append(c.lithology)

In [11]:
litho_list

['sable', 'remblais', 'limon', 'limons', 'schistes']

In [12]:
memoris_components = [Component({'lithology': 'remblais'}),
              Component({'lithology': 'limon'}),
              Component({'lithology': 'schistes'}),
              Component({'lithology': 'sable'}),
              Component({'lithology': 'anthropique'})
              ]
list_of_decors=[]
hatches = ['=', 'x', '.', 's', '=']
colours = ['#888888', '#882222', '#AAAAAA', '#CC22CC', '#CC2222']

for i in range(len(memoris_components)):
    if hasattr(memoris_components[i], 'colour'):
        c = memoris_components[i].colour
    else:
        c=colours[i]
    
    if hasattr(memoris_components[i], 'hatch'):
        h = memoris_components[i].hatch
    else:
        h=hatches[i]
        
    d = {'color': c,
         'hatch': h,
         'component': memoris_components[i],
         'width': 3}
    decor = Decor(d)
    list_of_decors.append(decor)
    
memoris_legend = Legend(list_of_decors)

In [13]:
boreholes

[<core.orm.BoreholeOrm>(Name=201, Length=1.7999999999999998, Diameter=75.0, Intervals=2),
 <core.orm.BoreholeOrm>(Name=205, Length=4.8, Diameter=75.0, Intervals=7),
 <core.orm.BoreholeOrm>(Name=208, Length=4.8, Diameter=75.0, Intervals=5),
 <core.orm.BoreholeOrm>(Name=212, Length=4.8, Diameter=75.0, Intervals=5),
 <core.orm.BoreholeOrm>(Name=207, Length=4.799999999999999, Diameter=75.0, Intervals=7),
 <core.orm.BoreholeOrm>(Name=214, Length=4.8, Diameter=75.0, Intervals=7),
 <core.orm.BoreholeOrm>(Name=217, Length=4.199999999999999, Diameter=75.0, Intervals=5),
 <core.orm.BoreholeOrm>(Name=221, Length=1.4, Diameter=75.0, Intervals=2),
 <core.orm.BoreholeOrm>(Name=223, Length=1.3, Diameter=75.0, Intervals=1),
 <core.orm.BoreholeOrm>(Name=225, Length=4.8, Diameter=75.0, Intervals=7),
 <core.orm.BoreholeOrm>(Name=224, Length=2.4, Diameter=75.0, Intervals=2),
 <core.orm.BoreholeOrm>(Name=219, Length=1.5, Diameter=75.0, Intervals=2),
 <core.orm.BoreholeOrm>(Name=220, Length=0.5, Diameter=75

In [14]:
components

{0: Component({'material': 'béton', 'grainsize': 'galets', 'Pollutant': 'naphtalène', 'lithology': 'sable', 'colour': 'rouge'}),
 1: Component({'lithology': 'remblais', 'colour': 'gris verdâtre'}),
 2: Component({'lithology': 'remblais', 'Pollutant': 'naphtalène'}),
 3: Component({'lithology': 'limon', 'colour': 'verdâtre', 'grainsize': 'grossiers'}),
 4: Component({'lithology': 'remblais', 'grainsize': 'grossiers', 'Pollutant': 'naphtalène'}),
 5: Component({'lithology': 'remblais', 'colour': 'gris'}),
 6: Component({'modifier': 'sableux', 'colour': 'rouge', 'lithology': 'sable'}),
 7: Component({'lithology': 'limon'}),
 8: Component({'lithology': 'sable', 'colour': 'blanc'}),
 9: Component({'grainsize': 'grossiers', 'lithology': 'sable', 'colour': 'rouge'}),
 10: Component({'lithology': 'remblais', 'modifier': 'sableuse', 'colour': 'sableuse'}),
 11: Component({'lithology': 'limons', 'colour': 'verdâtre'}),
 12: Component({'lithology': 'remblais', 'grainsize': 'grossiers'}),
 13: Com

In [15]:
len(boreholes), len(components)

(15, 45)

## Create a project and save data in a database

In [16]:
remove('tmp_files/tfe_orm_db.db')

In [17]:
engine = create_engine('sqlite:///tmp_files/tfe_orm_db.db', echo=True)

In [18]:
Base.metadata.create_all(engine)

2021-06-16 12:02:17,556 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-16 12:02:17,557 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Boreholes")
2021-06-16 12:02:17,558 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-16 12:02:17,559 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Boreholes")
2021-06-16 12:02:17,560 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-16 12:02:17,561 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Positions")
2021-06-16 12:02:17,562 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-16 12:02:17,563 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Positions")
2021-06-16 12:02:17,564 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-16 12:02:17,566 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Intervals")
2021-06-16 12:02:17,566 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-16 12:02:17,567 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Intervals")
2021-06-16 12:02:17,568 INFO sqlalchemy.engine.Engine 

In [19]:
Session = sessionmaker(bind=engine)
session = Session()

In [20]:
p = Project(session, legend=memoris_legend)
p.add_components(components)

2021-06-16 12:02:17,621 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-16 12:02:17,624 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-06-16 12:02:17,625 INFO sqlalchemy.engine.Engine [generated in 0.00116s] ()
2021-06-16 12:02:17,633 INFO sqlalchemy.engine.Engine INSERT INTO "Components" (id, description) VALUES (?, ?)
2021-06-16 12:02:17,634 INFO sqlalchemy.engine.Engine [generated in 0.00092s] ((0, 'Béton, galets, naphtalène, sable, rouge'), (1, 'Remblais, gris verdâtre'), (2, 'Remblais, naphtalène'), (3, 'Limon, verdâtre, grossiers'), (4, 'Remblais, grossiers, naphtalène'), (5, 'Remblais, gris'), (6, 'Sableux, rouge, sable'), (7, 'Limon')  ... displaying 10 of 45 total bound parameter sets ...  (43, 'Remblais, saturé'), (44, 'Limon, argileux, verdâtre'))
2021-06-16 12:02:17,636 INFO sqlalchemy.engine.Engine COMMIT
2021-06-16 12:02:17,640 INFO 

Add boreholes into the project

In [23]:
from utils.io import get_interval_list

In [26]:
get_interval_list(boreholes[0], lexicon='fr')

([Interval({'top': Position({'middle': 0.6, 'x': 152890.245758, 'y': 122596.47426, 'units': 'm'}), 'base': Position({'middle': 1.2, 'x': 152890.245758, 'y': 122596.47426, 'units': 'm'}), 'description': 'Remblais avec briques de construction couleur orange, gris, verdâtre  ', 'data': {}, 'components': [Component({'lithology': 'remblais', 'colour': 'gris verdâtre'})]}),
  Interval({'top': Position({'middle': 1.2, 'x': 152890.245758, 'y': 122596.47426, 'units': 'm'}), 'base': Position({'middle': 2.4, 'x': 152890.245758, 'y': 122596.47426, 'units': 'm'}), 'description': "Briquaille de déchets de construction et galets roulés - présence d'eau - légère odeur de naphtalène - Bloqué à 2,4 m sur caillasse (et béton ?)", 'data': {}, 'components': [Component({'material': 'béton', 'grainsize': 'galets', 'Pollutant': 'naphtalène'})]})],
 2.4)

In [21]:
for bh in boreholes:
    p.add_borehole(bh)

2021-06-16 12:02:17,649 INFO sqlalchemy.engine.Engine INSERT INTO "Boreholes" (id, length, diameter) VALUES (?, ?, ?)
2021-06-16 12:02:17,650 INFO sqlalchemy.engine.Engine [generated in 0.00129s] ('201', 1.7999999999999998, 75.0)
2021-06-16 12:02:17,652 INFO sqlalchemy.engine.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-06-16 12:02:17,653 INFO sqlalchemy.engine.Engine [generated in 0.00106s] ((0, 0.6, 0.6, 0.6, 152890.245758, 122596.47426), (1, 1.2, 1.2, 1.2, 152890.245758, 122596.47426), (2, 1.2, 1.2, 1.2, 152890.245758, 122596.47426), (3, 2.4, 2.4, 2.4, 152890.245758, 122596.47426))
2021-06-16 12:02:17,657 INFO sqlalchemy.engine.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description, top_id, base_id) VALUES (?, ?, ?, ?, ?, ?)
2021-06-16 12:02:17,658 INFO sqlalchemy.engine.Engine [generated in 0.00087s] ((0, '201', 0, 'Remblais avec briques de construction couleur orange, gris, verdâtre  ', 0, 1), (1, '201', 1, "Bri

TypeError: Cannot create a legend for empty component

In [22]:
components
#boreholes[0].intervals[0].components

{0: Component({'material': 'béton', 'grainsize': 'galets', 'Pollutant': 'naphtalène', 'lithology': 'sable', 'colour': 'rouge'}),
 1: Component({'lithology': 'remblais', 'colour': 'gris verdâtre'}),
 2: Component({'lithology': 'remblais', 'Pollutant': 'naphtalène'}),
 3: Component({'lithology': 'limon', 'colour': 'verdâtre', 'grainsize': 'grossiers'}),
 4: Component({'lithology': 'remblais', 'grainsize': 'grossiers', 'Pollutant': 'naphtalène'}),
 5: Component({'lithology': 'remblais', 'colour': 'gris'}),
 6: Component({'modifier': 'sableux', 'colour': 'rouge', 'lithology': 'sable'}),
 7: Component({'lithology': 'limon'}),
 8: Component({'lithology': 'sable', 'colour': 'blanc'}),
 9: Component({'grainsize': 'grossiers', 'lithology': 'sable', 'colour': 'rouge'}),
 10: Component({'lithology': 'remblais', 'modifier': 'sableuse', 'colour': 'sableuse'}),
 11: Component({'lithology': 'limons', 'colour': 'verdâtre'}),
 12: Component({'lithology': 'remblais', 'grainsize': 'grossiers'}),
 13: Com

In [ ]:
p.add_link_between_components_and_intervals(link_intv_comp)

In [ ]:
p.commit()
print('Boreholes in the project : ', len(p.boreholes))

In [ ]:
p.boreholes

In [ ]:
p.refresh(verbose=True)

In [ ]:
session.close()

# 3D Display

## Display one object

In [ ]:
bh = p.boreholes_3d
#print(bh.name, '\n', bh.intervals) # problem with top and base position --> kind of overlaying

In [ ]:
p.boreholes_3d[0].plot2d()

In [ ]:
p.boreholes_3d[0].plot3d()

In [ ]:
p.boreholes_3d[0].plot3d(x3d=True)

### Display project boreholes

In [ ]:
p.plot3d(labels_size=15, bg_color=["royalblue", "aliceblue"], window_size=(1300, 1000))

### Boreholes in 3D using X3D for interaction within the notebook